# GroupBy y Funciones de Agregación

Aprenderemos cómo usar los métodos GroupBy y Aggregate en un DataFrame. GroupBy te permite agrupar filas basándote en el valor de alguna columna. Por ejemplo, podrías agrupar datos de ventas por el día en que ocurrió la venta, o agrupar datos de clientes repetidos basándote en el nombre del cliente. Una vez que has realizado la operación GroupBy, puedes usar una función de agregación sobre esos datos. Una función de agregación agrega múltiples filas de datos en una única salida, como sumar las entradas o contar el número de entradas.

¡Veamos algunos ejemplos con un conjunto de datos de prueba!

In [ ]:
# Listamos los archivos disponibles en el sistema de archivos de Databricks
# dbutils.fs.ls() nos permite explorar directorios en DBFS (Databricks File System)
display(dbutils.fs.ls("dbfs:/databricks-datasets/data.gov/farmers_markets_geographic_data/data-001"))

In [ ]:
# Leemos un archivo CSV desde DBFS y lo cargamos en un DataFrame de Spark
# header=True indica que la primera fila contiene los nombres de las columnas
# inferSchema=True permite a Spark inferir automáticamente los tipos de datos de cada columna
df = spark.read.csv("dbfs:/databricks-datasets/data.gov/farmers_markets_geographic_data/data-001", header = True, inferSchema=True)

In [ ]:
# Mostramos el contenido del DataFrame en formato tabular
# display() es específico de Databricks y proporciona una visualización mejorada
display(df)

In [ ]:
# Cargamos un archivo CSV específico (market_data.csv) del directorio de mercados de agricultores
# Esta vez especificamos la ruta completa al archivo CSV
df = spark.read.csv("dbfs:/databricks-datasets/data.gov/farmers_markets_geographic_data/data-001/market_data.csv", header = True, inferSchema=True)

In [ ]:
# Visualizamos el DataFrame de datos de mercado
display(df)

In [ ]:
# Exploramos otro dataset: datos de compartición de bicicletas
# Listamos el contenido del directorio bikeSharing
display(dbutils.fs.ls("dbfs:/databricks-datasets/bikeSharing/data-001"))

In [ ]:
# Cargamos el dataset de compartición de bicicletas por día (day.csv)
# Este dataset contiene información agregada por día sobre el uso de bicicletas compartidas
df = spark.read.csv("dbfs:/databricks-datasets/bikeSharing/data-001/day.csv", header = True, inferSchema = True)

In [ ]:
# Mostramos el DataFrame de bicicletas compartidas
display(df)

In [ ]:
# Imprimimos el esquema del DataFrame para ver la estructura de los datos
# printSchema() muestra el nombre de cada columna, su tipo de dato y si acepta valores nulos
df.printSchema()

Leemos los datos de ventas de clientes

¡Agrupemos los datos por día laborable (workingday)!

In [ ]:
# Agrupamos el DataFrame por la columna "workingday"
# Esto crea un objeto GroupedData que nos permite aplicar funciones de agregación
df.groupBy("workingday")

Esto devuelve un objeto GroupedData, sobre el cual puedes llamar varios métodos de agregación

In [ ]:
# Calculamos el promedio de humedad (hum) agrupado por día laborable
# avg() calcula el valor promedio de la columna especificada para cada grupo
# show() muestra los resultados en consola
df.groupBy("workingday").avg("hum").show()

In [ ]:
# Contamos cuántos registros hay para cada día de la semana (Weekday)
# count() devuelve el número de filas en cada grupo
df.groupBy("Weekday").count().show()

In [ ]:
# Calculamos el valor máximo de todas las columnas numéricas agrupadas por mes (mnth)
# max() encuentra el valor máximo en cada columna para cada grupo
# take(4) solo toma las primeras 4 filas del resultado
display(df.groupBy("mnth").max().take(4))

In [ ]:
# Calculamos el valor mínimo de todas las columnas numéricas agrupadas por mes
# min() encuentra el valor mínimo en cada columna para cada grupo
display(df.groupBy("mnth").min().take(4))

In [ ]:
# Calculamos la suma de todas las columnas numéricas agrupadas por día festivo (holiday)
# sum() suma todos los valores en cada columna para cada grupo
# holiday: 0 = día no festivo, 1 = día festivo
display(df.groupBy("holiday").sum().take(2))

Consulta este enlace para más información sobre otros métodos:
http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark-sql-module

No todos los métodos necesitan una llamada a groupby, en su lugar puedes simplemente llamar al método generalizado .agg(), que llamará al agregado en todas las filas del DataFrame para la columna especificada. Puede tomar argumentos como una sola columna, o crear múltiples llamadas de agregación a la vez usando notación de diccionario.

Por ejemplo:

In [ ]:
# Calculamos el valor máximo de humedad en todo el DataFrame (sin agrupar)
# agg() permite aplicar funciones de agregación sin necesidad de groupBy
# {"hum":"max"} es notación de diccionario: columna -> función de agregación
df.agg({"hum":"max"}).show()

In [ ]:
# También podríamos haber hecho esto en el objeto groupBy:

In [ ]:
# Creamos un objeto agrupado por día laborable que reutilizaremos
grouped = df.groupBy("workingday")

In [ ]:
# Aplicamos la función de agregación max sobre la columna "hum" en el objeto agrupado
# Esto nos da el valor máximo de humedad para cada tipo de día (laborable o no)
grouped.agg({"hum":"max"}).show()

In [ ]:
# Importamos funciones de agregación desde pyspark.sql.functions
# Esto nos permite usar funciones más específicas como F.min() y F.max()
from pyspark.sql import functions as F

# Aplicamos múltiples funciones de agregación a la vez
# Calculamos tanto el mínimo como el máximo de humedad para cada grupo
grouped.agg(F.min("hum"), F.max("hum")).show()

In [ ]:
# Convertimos el DataFrame de Spark a un DataFrame de Pandas
# Pandas es útil para análisis y visualización en memoria con datasets pequeños
# IMPORTANTE: Solo usa esto con datos que caben en memoria
pandas_df = df.toPandas()

In [ ]:
# Mostramos las primeras 5 filas del DataFrame de Pandas
# head() es un método estándar de Pandas
pandas_df.head()

## Funciones
Hay una variedad de funciones que puedes importar desde pyspark.sql.functions. Consulta la documentación para ver la lista completa disponible:
http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions

In [ ]:
# Importamos funciones especializadas para cálculos estadísticos
# countDistinct: cuenta valores únicos
# avg: calcula el promedio
# stddev: calcula la desviación estándar
# Estas funciones son óptimas para Big Data porque están diseñadas para trabajar con particiones distribuidas
from pyspark.sql.functions import countDistinct, avg, stddev

In [ ]:
# Contamos cuántos valores distintos (únicos) hay en la columna "registered"
# countDistinct() elimina duplicados y cuenta solo valores únicos
df.select(countDistinct("registered")).show()

A menudo querrás cambiar el nombre de la columna resultante, usa el método .alias() para esto:

In [ ]:
# Renombramos la columna del resultado usando alias()
# Esto hace que la salida sea más legible y descriptiva
df.select(countDistinct("registered").alias("registros_unicos")).show()

In [ ]:
# Calculamos el promedio (media) de la columna "hum" (humedad)
# avg() suma todos los valores y los divide por el número de filas
df.select(avg("hum")).show()

In [ ]:
# Calculamos la desviación estándar de la columna "hum"
# stddev() mide la dispersión de los datos respecto a la media
# Es útil para entender la variabilidad de los datos
df.select(stddev("hum")).show()

¡Eso es mucha precisión en los dígitos! Usemos format_number para arreglar eso!

In [ ]:
# Importamos format_number para formatear números con decimales específicos
from pyspark.sql.functions import format_number

In [ ]:
# Calculamos la desviación estándar y la guardamos en un DataFrame
# Le damos el alias "std" para simplificar el nombre de la columna
hum_std = df.select(stddev("hum").alias("std"))

In [ ]:
# Verificamos el tipo de objeto que hemos creado
# Sigue siendo un DataFrame de PySpark, no un valor numérico simple
type(hum_std)

In [ ]:
# Mostramos el DataFrame con la desviación estándar
hum_std.show()

In [ ]:
# Formateamos el número para mostrar solo 2 decimales
# format_number(columna, número_de_decimales)
# Esto hace que los resultados sean más legibles
hum_std.select(format_number("std", 2)).show()

## Ordenar (Order By)

Puedes ordenar fácilmente con el método orderBy:

In [ ]:
# Ordenamos el DataFrame por la columna "registered" en orden ascendente (por defecto)
# Los valores más pequeños aparecen primero
display(df.orderBy("registered"))

In [ ]:
# Ordenamos el DataFrame en orden descendente usando .desc()
# Los valores más grandes aparecen primero
display(df.orderBy(df["registered"].desc()))

La mayoría de las funciones básicas que esperarías están disponibles, ¡así que asegúrate de consultar la documentación!